In [ ]:
import requests
import tables
import io
import scipy
import numpy
from scipy import sparse
import pandas

## Query data available

In [ ]:
url = "https://scrap.caltech.edu/references"
response = requests.get(url, verify=False)

In [ ]:
reference_names = [ref["name"] for ref in response.json()["references"]]
print(reference_names)

In [ ]:
url = "https://scrap.caltech.edu/samples"
response = requests.get(url, verify=False)

In [ ]:
sample_names = [sample["name"] for sample in response.json()["samples"]]
print(sample_names)

## Access data programmatically

In [ ]:
url = "https://scrap.caltech.edu/transcripts_file?id=5b9ad17bab67490fee3ca30c&file=filtered_gene_bc_matrices_h5.h5&sample_id=5b9ad13c27614e14ddfc919c"
response = requests.get(url, verify=False)

In [ ]:
h5file = tables.open_file("tmp.h5", driver="H5FD_CORE",
                          driver_core_image=response.content,
                          driver_core_backing_store=0)

In [ ]:
barcodes = [barcode.decode("utf-8") for barcode in h5file.root.hg19.barcodes.read()]
print(barcodes[0:10])

In [ ]:
gene_names = [gene.decode("utf-8") for gene in h5file.root.hg19.gene_names.read()]
print(gene_names[0:10])

In [ ]:
h5file.root.hg19.data

In [ ]:
data = sparse.csc_matrix((h5file.root.hg19.data.read(), h5file.root.hg19.indices.read(), h5file.root.hg19.indptr.read()), shape=h5file.root.hg19.shape.read())

In [ ]:
data[0:100,0:100].toarray()

In [ ]:
dataframe = pandas.DataFrame(data.toarray(), index=gene_names, columns=barcodes)

In [ ]:
dataframe.head()